论文阅读 + 复现：（LRCNs）Long-term Recurrent Convolutional Networks for Visual Recognition and Description:


https://blog.csdn.net/qq_36627158/article/details/114026519

GitHub:

https://github.com/BizhuWu/LRCN_PyTorch

In [1]:
# clone git repo
!git clone https://github.com/BizhuWu/LRCN_PyTorch
!mv LRCN_PyTorch/* .
!rm -rf LRCN_PyTorch/

Cloning into 'LRCN_PyTorch'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 36 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [2]:
# Download UCF101 dataset and split list
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

--2021-06-08 05:29:19--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  11.5MB/s    in 9m 43s  

2021-06-08 05:39:03 (11.3 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]

--2021-06-08 05:39:03--  https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 113943 (111K) [application/zip]
Saving to: ‘UCF101TrainTestSplits-RecognitionTask.zip’

UCF

In [4]:
# Extract and organize video data and split list
!mkdir -p data/UCF101/UCF-101/
!unrar e UCF101.rar data/UCF101/UCF-101/
import os
for files in os.listdir('data/UCF101/UCF-101/'):
    foldername = files.split('_')[1]
    os.system("mkdir -p data/UCF101/UCF-101/" + foldername)
    os.system("mv data/UCF101/UCF-101/"+ files + " data/UCF101/UCF-101/" + foldername)
# !rm data/UCF101/UCF-101/*.avi

!mkdir -p data/UCF101TrainTestSplits-RecognitionTask
!unzip UCF101TrainTestSplits-RecognitionTask.zip -d data/UCF101TrainTestSplits-RecognitionTask

Archive:  UCF101TrainTestSplits-RecognitionTask.zip
   creating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist01.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist02.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist03.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist01.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist02.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist03.txt  


In [1]:
!nvidia-smi
!which python
!which apt-get
!which pip

Tue Jun  8 11:44:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   47C    P2    54W / 280W |    247MiB / 11178MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Install dependencies for video module in pytorch to work
# ! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

  Using cached av-6.2.0.tar.gz (2.0 MB)
  Created wheel for av: filename=av-6.2.0-cp37-cp37m-linux_x86_64.whl size=5120008 sha256=26de819f8937ceaccce65b2d65fb18c817000d98b26172e2708c415314d67634
  Stored in directory: /home/chihyuan/.cache/pip/wheels/bd/83/e3/93ca73311d72bebad4a806dc9a337f7d9e17dc5ec3fd4b58a8
Successfully built av


In [7]:
from LoadUCF101DataByTorch import trainset_loader, testset_loader
from model import LRCN
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt



EPOCH = 10
LEARNING_RATE = 0.003
MOMENTUM = 0.9
GAMMA = 0.5
STEP_SIZE = 1

print("HELLO")

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)



model = LRCN().to(device)



optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    momentum=MOMENTUM
)
# scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer,
#     step_size=STEP_SIZE,
#     gamma=GAMMA
# )



def save_checkpoint(path, model, optimizer):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, path)



def train(epoch):
    iteration = 0
    loss_plt=[]


    for i in range(epoch):
        model.train()
        # print('current lr', scheduler.get_last_lr())
        
        for index, data in enumerate(trainset_loader):
            video_clips, label = data

            video_clips = video_clips.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            output = model(video_clips)

            loss = F.cross_entropy(output, label)

            loss_plt.append(loss.item())

            loss.backward()
            optimizer.step()

            iteration += 1

            print("Epoch:", i, "/", epoch-1, "\tIteration:", index, "/", len(trainset_loader)-1, "\tLoss: " + str(loss.item()))
            with open('log.txt', 'a') as f:
                f.write("Epoch: " + str(i) + "/" + str(epoch-1) + "\tIteration:" + str(index) + "/" + str(len(trainset_loader)-1) + "\tLoss: " + str(loss.item()) + "\n")

        save_checkpoint('model/checkpoint-%i.pth' % iteration, model, optimizer)

        test(i)

        # scheduler.step()

    save_checkpoint('model/checkpoint-%i.pth' % iteration, model, optimizer)

    plt.figure()
    plt.plot(loss_plt)
    plt.title('Loss')
    plt.xlabel('Iteration')
    plt.ylabel('')
    plt.show()



def test(i_epoch):
    model.eval()

    correct = 0

    with torch.no_grad():
        for index, data in enumerate(testset_loader):
            video_clips, label = data

            video_clips = video_clips.to(device)
            label = label.to(device)

            output = model(video_clips)

            max_value, max_index = output.max(1, keepdim=True)
            correct += max_index.eq(label.view_as(max_index)).sum().item()

    print("Accuracy: " + str(correct * 1.0 * 100 / len(testset_loader.dataset)))
    with open('log.txt', 'a') as f:
        f.write("Epoch " + str(i_epoch) + "'s Accuracy: " + str(correct * 1.0 * 100 / len(testset_loader.dataset)) + "\n")



HELLO
cuda


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/chihyuan/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

/home/chihyuan/anaconda3/envs/ntupac21s/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
train(EPOCH)

Epoch: 0 / 9 	Iteration: 0 / 3198 	Loss: 0.6280677318572998
Epoch: 0 / 9 	Iteration: 1 / 3198 	Loss: 0.712385356426239
Epoch: 0 / 9 	Iteration: 2 / 3198 	Loss: 0.6944305896759033
Epoch: 0 / 9 	Iteration: 3 / 3198 	Loss: 0.6294537782669067
Epoch: 0 / 9 	Iteration: 4 / 3198 	Loss: 0.7105886936187744
Epoch: 0 / 9 	Iteration: 5 / 3198 	Loss: 0.7280457019805908
Epoch: 0 / 9 	Iteration: 6 / 3198 	Loss: 0.7238345742225647
Epoch: 0 / 9 	Iteration: 7 / 3198 	Loss: 0.7378685474395752
Epoch: 0 / 9 	Iteration: 8 / 3198 	Loss: 0.6119613647460938
Epoch: 0 / 9 	Iteration: 9 / 3198 	Loss: 0.654222309589386
Epoch: 0 / 9 	Iteration: 10 / 3198 	Loss: 0.5879912972450256
Epoch: 0 / 9 	Iteration: 11 / 3198 	Loss: 0.6622620224952698
Epoch: 0 / 9 	Iteration: 12 / 3198 	Loss: 0.687736988067627
Epoch: 0 / 9 	Iteration: 13 / 3198 	Loss: 0.706867516040802
Epoch: 0 / 9 	Iteration: 14 / 3198 	Loss: 0.6163299679756165
Epoch: 0 / 9 	Iteration: 15 / 3198 	Loss: 0.7072651982307434
Epoch: 0 / 9 	Iteration: 16 / 3198 	Lo

Epoch: 0 / 9 	Iteration: 135 / 3198 	Loss: 0.6194303631782532
Epoch: 0 / 9 	Iteration: 136 / 3198 	Loss: 0.7112469673156738
Epoch: 0 / 9 	Iteration: 137 / 3198 	Loss: 0.8337692022323608
Epoch: 0 / 9 	Iteration: 138 / 3198 	Loss: 0.4754054844379425
Epoch: 0 / 9 	Iteration: 139 / 3198 	Loss: 0.6686245203018188
Epoch: 0 / 9 	Iteration: 140 / 3198 	Loss: 0.6038534641265869
Epoch: 0 / 9 	Iteration: 141 / 3198 	Loss: 0.6420580148696899
Epoch: 0 / 9 	Iteration: 142 / 3198 	Loss: 0.7502979040145874
Epoch: 0 / 9 	Iteration: 143 / 3198 	Loss: 0.7080347537994385
Epoch: 0 / 9 	Iteration: 144 / 3198 	Loss: 0.5904120206832886
Epoch: 0 / 9 	Iteration: 145 / 3198 	Loss: 0.608615517616272
Epoch: 0 / 9 	Iteration: 146 / 3198 	Loss: 0.6761999130249023
Epoch: 0 / 9 	Iteration: 147 / 3198 	Loss: 0.7046387195587158
Epoch: 0 / 9 	Iteration: 148 / 3198 	Loss: 0.7480195164680481
Epoch: 0 / 9 	Iteration: 149 / 3198 	Loss: 0.656624436378479
Epoch: 0 / 9 	Iteration: 150 / 3198 	Loss: 0.6557757258415222
Epoch: 0 /

Epoch: 0 / 9 	Iteration: 268 / 3198 	Loss: 0.687192976474762
Epoch: 0 / 9 	Iteration: 269 / 3198 	Loss: 0.6662087440490723
Epoch: 0 / 9 	Iteration: 270 / 3198 	Loss: 0.5742968916893005
Epoch: 0 / 9 	Iteration: 271 / 3198 	Loss: 0.6239973902702332
Epoch: 0 / 9 	Iteration: 272 / 3198 	Loss: 0.6309476494789124
Epoch: 0 / 9 	Iteration: 273 / 3198 	Loss: 0.6304685473442078
Epoch: 0 / 9 	Iteration: 274 / 3198 	Loss: 0.6502752304077148
Epoch: 0 / 9 	Iteration: 275 / 3198 	Loss: 0.8116390109062195
Epoch: 0 / 9 	Iteration: 276 / 3198 	Loss: 0.5943416357040405
Epoch: 0 / 9 	Iteration: 277 / 3198 	Loss: 0.6871936321258545
Epoch: 0 / 9 	Iteration: 278 / 3198 	Loss: 0.6438028812408447
Epoch: 0 / 9 	Iteration: 279 / 3198 	Loss: 0.5956559777259827
Epoch: 0 / 9 	Iteration: 280 / 3198 	Loss: 0.7002571821212769
Epoch: 0 / 9 	Iteration: 281 / 3198 	Loss: 0.6538786888122559
Epoch: 0 / 9 	Iteration: 282 / 3198 	Loss: 0.6185054183006287
Epoch: 0 / 9 	Iteration: 283 / 3198 	Loss: 0.6901701092720032
Epoch: 0 

Epoch: 0 / 9 	Iteration: 401 / 3198 	Loss: 0.6195880770683289
Epoch: 0 / 9 	Iteration: 402 / 3198 	Loss: 0.593579888343811
Epoch: 0 / 9 	Iteration: 403 / 3198 	Loss: 0.7519869208335876
Epoch: 0 / 9 	Iteration: 404 / 3198 	Loss: 0.6707058548927307
Epoch: 0 / 9 	Iteration: 405 / 3198 	Loss: 0.5841778516769409
Epoch: 0 / 9 	Iteration: 406 / 3198 	Loss: 0.7192482948303223
Epoch: 0 / 9 	Iteration: 407 / 3198 	Loss: 0.5435344576835632
Epoch: 0 / 9 	Iteration: 408 / 3198 	Loss: 0.6282426118850708
Epoch: 0 / 9 	Iteration: 409 / 3198 	Loss: 0.8629629015922546
Epoch: 0 / 9 	Iteration: 410 / 3198 	Loss: 0.6442152261734009
Epoch: 0 / 9 	Iteration: 411 / 3198 	Loss: 0.5997463464736938
Epoch: 0 / 9 	Iteration: 412 / 3198 	Loss: 0.6795791387557983
Epoch: 0 / 9 	Iteration: 413 / 3198 	Loss: 0.5385199189186096
Epoch: 0 / 9 	Iteration: 414 / 3198 	Loss: 0.6385663151741028
Epoch: 0 / 9 	Iteration: 415 / 3198 	Loss: 0.6321070194244385
Epoch: 0 / 9 	Iteration: 416 / 3198 	Loss: 0.6154075264930725
Epoch: 0 

Epoch: 0 / 9 	Iteration: 534 / 3198 	Loss: 0.5819803476333618
Epoch: 0 / 9 	Iteration: 535 / 3198 	Loss: 0.6408758759498596
Epoch: 0 / 9 	Iteration: 536 / 3198 	Loss: 0.5769132375717163
Epoch: 0 / 9 	Iteration: 537 / 3198 	Loss: 0.40746819972991943
Epoch: 0 / 9 	Iteration: 538 / 3198 	Loss: 0.6375983357429504
Epoch: 0 / 9 	Iteration: 539 / 3198 	Loss: 0.466685026884079
Epoch: 0 / 9 	Iteration: 540 / 3198 	Loss: 0.5198494791984558
Epoch: 0 / 9 	Iteration: 541 / 3198 	Loss: 0.6098594665527344
Epoch: 0 / 9 	Iteration: 542 / 3198 	Loss: 0.5794055461883545
Epoch: 0 / 9 	Iteration: 543 / 3198 	Loss: 0.6166967153549194
Epoch: 0 / 9 	Iteration: 544 / 3198 	Loss: 0.46345722675323486
Epoch: 0 / 9 	Iteration: 545 / 3198 	Loss: 0.6750487685203552
Epoch: 0 / 9 	Iteration: 546 / 3198 	Loss: 0.870153546333313
Epoch: 0 / 9 	Iteration: 547 / 3198 	Loss: 0.5668254494667053
Epoch: 0 / 9 	Iteration: 548 / 3198 	Loss: 0.5680418610572815
Epoch: 0 / 9 	Iteration: 549 / 3198 	Loss: 0.5886456966400146
Epoch: 0

...

Epoch: 0 / 9 	Iteration: 1455 / 3198 	Loss: 0.5926836729049683
Epoch: 0 / 9 	Iteration: 1456 / 3198 	Loss: 0.41661927103996277
Epoch: 0 / 9 	Iteration: 1457 / 3198 	Loss: 0.460247665643692
Epoch: 0 / 9 	Iteration: 1458 / 3198 	Loss: 0.45755866169929504
Epoch: 0 / 9 	Iteration: 1459 / 3198 	Loss: 0.5749346613883972
Epoch: 0 / 9 	Iteration: 1460 / 3198 	Loss: 0.5293717980384827
Epoch: 0 / 9 	Iteration: 1461 / 3198 	Loss: 0.46887049078941345
Epoch: 0 / 9 	Iteration: 1462 / 3198 	Loss: 0.482517272233963
Epoch: 0 / 9 	Iteration: 1463 / 3198 	Loss: 0.4907169044017792
Epoch: 0 / 9 	Iteration: 1464 / 3198 	Loss: 0.47411975264549255
Epoch: 0 / 9 	Iteration: 1465 / 3198 	Loss: 0.503982424736023
Epoch: 0 / 9 	Iteration: 1466 / 3198 	Loss: 0.4682961404323578
Epoch: 0 / 9 	Iteration: 1467 / 3198 	Loss: 0.4914393723011017
Epoch: 0 / 9 	Iteration: 1468 / 3198 	Loss: 0.6290578842163086
Epoch: 0 / 9 	Iteration: 1469 / 3198 	Loss: 0.5118680000305176
Epoch: 0 / 9 	Iteration: 1470 / 3198 	Loss: 0.54604607

Epoch: 0 / 9 	Iteration: 1585 / 3198 	Loss: 0.41857266426086426
Epoch: 0 / 9 	Iteration: 1586 / 3198 	Loss: 0.47787293791770935
Epoch: 0 / 9 	Iteration: 1587 / 3198 	Loss: 0.44986435770988464
Epoch: 0 / 9 	Iteration: 1588 / 3198 	Loss: 0.42410996556282043
Epoch: 0 / 9 	Iteration: 1589 / 3198 	Loss: 0.501640796661377
Epoch: 0 / 9 	Iteration: 1590 / 3198 	Loss: 0.33741772174835205
Epoch: 0 / 9 	Iteration: 1591 / 3198 	Loss: 0.4895407259464264
Epoch: 0 / 9 	Iteration: 1592 / 3198 	Loss: 0.4808741807937622
Epoch: 0 / 9 	Iteration: 1593 / 3198 	Loss: 0.5115897059440613
Epoch: 0 / 9 	Iteration: 1594 / 3198 	Loss: 0.41387397050857544
Epoch: 0 / 9 	Iteration: 1595 / 3198 	Loss: 0.3827737271785736
Epoch: 0 / 9 	Iteration: 1596 / 3198 	Loss: 0.5427954792976379
Epoch: 0 / 9 	Iteration: 1597 / 3198 	Loss: 0.550295352935791
Epoch: 0 / 9 	Iteration: 1598 / 3198 	Loss: 0.5319672226905823
Epoch: 0 / 9 	Iteration: 1599 / 3198 	Loss: 0.47551825642585754
Epoch: 0 / 9 	Iteration: 1600 / 3198 	Loss: 0.4169

Epoch: 0 / 9 	Iteration: 1715 / 3198 	Loss: 0.45924487709999084
Epoch: 0 / 9 	Iteration: 1716 / 3198 	Loss: 0.5202039480209351
Epoch: 0 / 9 	Iteration: 1717 / 3198 	Loss: 0.5886806845664978
Epoch: 0 / 9 	Iteration: 1718 / 3198 	Loss: 0.4613877832889557
Epoch: 0 / 9 	Iteration: 1719 / 3198 	Loss: 0.5174980759620667
Epoch: 0 / 9 	Iteration: 1720 / 3198 	Loss: 0.4968443512916565
Epoch: 0 / 9 	Iteration: 1721 / 3198 	Loss: 0.4425659477710724
Epoch: 0 / 9 	Iteration: 1722 / 3198 	Loss: 0.4235802888870239
Epoch: 0 / 9 	Iteration: 1723 / 3198 	Loss: 0.38313689827919006
Epoch: 0 / 9 	Iteration: 1724 / 3198 	Loss: 0.4690004289150238
Epoch: 0 / 9 	Iteration: 1725 / 3198 	Loss: 0.5075371265411377
Epoch: 0 / 9 	Iteration: 1726 / 3198 	Loss: 0.5155350565910339
Epoch: 0 / 9 	Iteration: 1727 / 3198 	Loss: 0.5454737544059753
Epoch: 0 / 9 	Iteration: 1728 / 3198 	Loss: 0.44543957710266113
Epoch: 0 / 9 	Iteration: 1729 / 3198 	Loss: 0.40590840578079224
Epoch: 0 / 9 	Iteration: 1730 / 3198 	Loss: 0.49352

Epoch: 0 / 9 	Iteration: 1845 / 3198 	Loss: 0.418663889169693
Epoch: 0 / 9 	Iteration: 1846 / 3198 	Loss: 0.46152859926223755
Epoch: 0 / 9 	Iteration: 1847 / 3198 	Loss: 0.5753265619277954
Epoch: 0 / 9 	Iteration: 1848 / 3198 	Loss: 0.4004272520542145
Epoch: 0 / 9 	Iteration: 1849 / 3198 	Loss: 0.39392369985580444
Epoch: 0 / 9 	Iteration: 1850 / 3198 	Loss: 0.42932915687561035
Epoch: 0 / 9 	Iteration: 1851 / 3198 	Loss: 0.5958582758903503
Epoch: 0 / 9 	Iteration: 1852 / 3198 	Loss: 0.5155505537986755
Epoch: 0 / 9 	Iteration: 1853 / 3198 	Loss: 0.46172404289245605
Epoch: 0 / 9 	Iteration: 1854 / 3198 	Loss: 0.49796056747436523
Epoch: 0 / 9 	Iteration: 1855 / 3198 	Loss: 0.4129383862018585
Epoch: 0 / 9 	Iteration: 1856 / 3198 	Loss: 0.3816685974597931
Epoch: 0 / 9 	Iteration: 1857 / 3198 	Loss: 0.4434271454811096
Epoch: 0 / 9 	Iteration: 1858 / 3198 	Loss: 0.4329487085342407
Epoch: 0 / 9 	Iteration: 1859 / 3198 	Loss: 0.5332974195480347
Epoch: 0 / 9 	Iteration: 1860 / 3198 	Loss: 0.49520

Epoch: 0 / 9 	Iteration: 1975 / 3198 	Loss: 0.4642261862754822
Epoch: 0 / 9 	Iteration: 1976 / 3198 	Loss: 0.4373626708984375
Epoch: 0 / 9 	Iteration: 1977 / 3198 	Loss: 0.40787696838378906
Epoch: 0 / 9 	Iteration: 1978 / 3198 	Loss: 0.4496198296546936
Epoch: 0 / 9 	Iteration: 1979 / 3198 	Loss: 0.374875545501709
Epoch: 0 / 9 	Iteration: 1980 / 3198 	Loss: 0.4406929314136505
Epoch: 0 / 9 	Iteration: 1981 / 3198 	Loss: 0.5082820653915405
Epoch: 0 / 9 	Iteration: 1982 / 3198 	Loss: 0.391171395778656
Epoch: 0 / 9 	Iteration: 1983 / 3198 	Loss: 0.4956955909729004
Epoch: 0 / 9 	Iteration: 1984 / 3198 	Loss: 0.3888908326625824
Epoch: 0 / 9 	Iteration: 1985 / 3198 	Loss: 0.4393059313297272
Epoch: 0 / 9 	Iteration: 1986 / 3198 	Loss: 0.40071535110473633
Epoch: 0 / 9 	Iteration: 1987 / 3198 	Loss: 0.47450804710388184
Epoch: 0 / 9 	Iteration: 1988 / 3198 	Loss: 0.504504919052124
Epoch: 0 / 9 	Iteration: 1989 / 3198 	Loss: 0.48264652490615845
Epoch: 0 / 9 	Iteration: 1990 / 3198 	Loss: 0.42518356